## Spork

I created this module because I wanted a simple way to describe how to render pydantic models. I almost always repeat the same logic across pydantic models. Maybe I even want the same on dataclasses.

To explore this space a bit, I've created a `View` and an `AutoView`. This lets you update fields and have them update within the notebook. This notebook is best viewed in a live running notebook environment (jupyterlab, VS Code, Colab, etc.)

In [1]:
from spork import View

class BigEmoji(View):
    emoji: str = "😄"
    
    def render(self):
        return f"<span style='font-size: 2em'>{self.emoji}</span>"

face = BigEmoji()
face.display()
face.emoji = "🥰"
face.update()

BigEmoji(display_id='7974ecc7-bddb-4691-9fd9-07b1c1306062', emoji='🥰')

In [2]:
from spork import AutoView
from pydantic import BaseModel
from time import sleep

class Countdown(AutoView):
    time: float = 10
    delta: float = .01

    def start(self):
        while self.time > 0:
            self.time -= self.delta
            if self.time < 0:
                self.time = 0
            sleep(self.delta)

    def render(self):
        return f"{self.time:.2f} seconds left"

countdown = Countdown(time=1)

countdown.display()
countdown.start()

Countdown(display_id='4359def9-47bf-4a8f-bd54-2f755edec227', time=0, delta=0.01)

### Returning HTML

You can return raw HTML to render a view for your model as well.

In [3]:
from spork import AutoView
from time import sleep

class CountdownPretty(AutoView):
    time: float = 10
    delta: float = .01

    def start(self):
        while self.time > 0:
            self.time -= self.delta
            if self.time < 0:
                self.time = 0
            sleep(self.delta)

    def render(self):
        color = "#4fd953"
        if self.time <= 0:
            color = "#d9534f"
        
        return f'''
    <div style="padding: 10px; background-color: #f0f0f0; color: black; border-radius: 5px; text-align: center; width: 250px; margin: 10px auto;">
        <span style="font-size: 20px; font-weight: bold;">Time Left:</span>
        <div style="font-size: 24px; color: {color}; font-family: 'Courier New', monospace;">
            {self.time:0.2f}
        </div>
        <div style="font-size: 20px; color: black;">seconds</div>
    </div>
        '''

countdown = CountdownPretty(time=1)

countdown.display()
countdown.start()

CountdownPretty(display_id='611ff5c9-23f7-44aa-a497-ec55b31e097a', time=0, delta=0.01)

## VDOM or anything with a `to_html`

You can also use the composable components from VDOM to render your objects.

In [4]:
from spork import AutoView
from time import sleep
from vdom.helpers import div, span

class CountdownPretty(AutoView):
    time: float = 10
    delta: float = 0.01

    def start(self):
        while self.time > 0:
            self.time -= self.delta
            if self.time < 0:
                self.time = 0
            sleep(self.delta)

    def render(self):
        return div(
            style={
                "padding": "10px",
                "backgroundColor": "#f0f0f0",
                "borderRadius": "5px",
                "textAlign": "center",
                "width": "250px",
                "margin": "10px auto",
                "color": "black",
                "fontFamily": "'Courier New', monospace"
            },
            children=[
                span(
                    "Time Left:",
                    style={
                        "fontSize": "20px",
                        "fontWeight": "bold"
                    }
                ),
                div(
                    f"{self.time:02.2f}",
                    style={
                        "fontFamily": "'Courier New', monospace",
                        "fontSize": "24px",
                        "color": "#4fd953" if self.time > 0 else "#d9534f"
                    }
                ),
                div(
                    "seconds",
                    style={
                        "fontSize": "20px",
                        "color": "black"
                    }
                )
            ]
        )

countdown = CountdownPretty(time=3)
countdown.display()
countdown.start()

CountdownPretty(display_id='b1d2b90c-0fad-4b51-aa9b-49f18f41fc0c', time=0, delta=0.01)

You can also use existing pydantic models by creating a new class that inherits from the original and the `AutoView`:

In [5]:
from spork import AutoView
from pydantic import BaseModel

class Record(BaseModel):
    name: str
    age: int

class RecordView(Record, AutoView):
    def render(self):
        return f"<b>{self.name}</b> is {self.age} years old."


rv = RecordView(name="Kyle", age=35)
rv.display()

RecordView(display_id='df347d47-12e5-4a66-b6d0-1d4cc9b2732b', name='Kyle', age=35)

There are also decorators that allow for using `render` without having to inherit from the `View`.



In [6]:
from spork import renderable

@renderable
class Record(BaseModel):
    name: str
    age: int

    def render(self):
        return f"<b>{self.name}</b> is {self.age} years old."
    
display(Record(name="Kyle", age=35))


Record(name='Kyle', age=35)